- Name: Raisun Lakra
- Objective: Develop a face recognition system using opencv.
- Dataset: LFW - People (Face Recognition)
- Dataset link: https://www.kaggle.com/datasets/atulanandjha/lfwpeople

In [8]:
import cv2
import dlib
import numpy as np
import os
import shutil
import random
import face_recognition
import pickle

In [2]:
# video_cap = cv2.VideoCapture(0)
# while True:
#     ret, image = video_cap.read()
#     cv2.imshow("Live Video - Face Recognition", image)
#     if cv2.waitKey == ord("a"):
#         break
# video_cap.release()
# TODO : Resize image

### FACE DETECTION

In [25]:
class FaceDetector():
    def __init__(self, faceCascadePath):
        """ Apply Haar Cascade to detect faces """
        self.faceCascade = cv2.CascadeClassifier(faceCascadePath)
        
        # Check if the cascade is loaded properly
        if self.faceCascade.empty():
            raise ValueError(f"Error loading cascade file: {faceCascadePath}")

    def detect(self, image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)):
        """ Return x, y, width, height of detected faces """
        rects = self.faceCascade.detectMultiScale(image, scaleFactor=scaleFactor, minNeighbors=minNeighbors, minSize=minSize)
        return rects

def detect_live():
    # Start video capture from webcam
    video_cap = cv2.VideoCapture(0)
    
    frontal_face_Cascade_Path = "/home/user/Desktop/internship project/Cyfuture-face-recognition-opencv/face-recog-env/lib/python3.13/site-packages/cv2/data/haarcascade_frontalface_default.xml"
    fd = FaceDetector(frontal_face_Cascade_Path)

    while True:
        ret, image = video_cap.read()
        if not ret:
            print("Failed to capture image")
            break

        # Resize the image to a fixed width and height - required for linux to reshape the image as linux is not built for general purpose task. Its default image size too small.
        image_resized = cv2.resize(image, (640, 480))

        # Convert rgb image to grayscale image
        gray_image = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = fd.detect(gray_image)

        # Draw rectangles around detected faces
        for x, y, w, h in faces:
            cv2.rectangle(image_resized, (x, y), (x + w, y + h), (127, 255, 0), 2)

        # Show the resized video feed
        cv2.imshow("Live Video - Face Recognition", image_resized)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord("q") or cv2.getWindowProperty("Live Video - Face Recognition", cv2.WND_PROP_VISIBLE) < 1:
            break

    # Release resources
    video_cap.release()
    cv2.destroyAllWindows()

In [26]:
# Uncomment to Run the function
# detect_live()

Failed to capture image


[ WARN:0@8946.616] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@8946.616] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


### **RESULT**: System is unable to detect face when background is not good like getting sun light from background or in dark room (means low light source). Otherwise for plane bachground it is working good.

# PREPROCESSING

In [5]:
# ingest.py

# import random
# import os
# import shutil

def extract_images():
    datasets_path = "../Datasets/images/lfw_funneled/"
    processed_image_path = "../Datasets/processed_images/"
    
    counter = 0
    for person in os.listdir(datasets_path):
        folder_path = os.path.join(datasets_path, person)
        if os.path.isdir(folder_path):
            photos = [photo for photo in os.listdir(folder_path) if photo.lower().endswith(('.jpg','.png', '.jpeg'))]
            if 10 <= len(photos) <= 20:
                selected_photos = photos
            elif len(photos) > 20:
                selected_photos = random.sample(photos, 20)
            else:
                continue
    
            counter += 1
            os.makedirs(processed_image_path, exist_ok=True)
            os.makedirs(os.path.join(processed_image_path, person), exist_ok=True)
    
            for image in selected_photos:
                src = os.path.join(folder_path, image)
                dst = os.path.join(processed_image_path, person, image)
                shutil.copy(src, dst)
        else:
            print(f"{folder_path} is not a directory.")
    print("{} new folders will be created.".format(counter))

In [1]:
# till now we have extracted required images and implemented face detection

In [2]:
# Now we have to encode the images and apply face recognition

### ENCODE IMAGES

In [11]:
DATASET_PATH = "../Datasets/processed_images/"
ENCODINGS_FILE = "face_encodings.pkl"

def encode_faces(dataset_path):
    known_encodings = []
    known_names = []

    for person_name in os.listdir(dataset_path):
        person_dir = os.path.join(dataset_path, person_name)

        if not os.path.isdir(person_dir):
            continue

        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            print(f"Processing {image_path}")

            image = cv2.imread(image_path)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Detect face locations
            face_locations = face_recognition.face_locations(rgb_image)

            # Get face encodings
            encodings = face_recognition.face_encodings(rgb_image, face_locations)

            for encoding in encodings:
                known_encodings.append(encoding)
                known_names.append(person_name)

    # Save encodings
    data = {"encodings": known_encodings, "names": known_names}
    with open(ENCODINGS_FILE, "wb") as f:
        pickle.dump(data, f)

    print("Encoding complete and saved.")

# Run the encoding function
encode_faces(DATASET_PATH)

Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0002.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0011.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0009.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0010.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0003.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0001.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0004.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0005.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0006.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0008.jpg
Processing ../Datasets/processed_images/Richard_Gephardt/Richard_Gephardt_0007.jpg
Processing ../Datasets/processed_images/Atal_Bihari_Vajpayee/Atal_Bihari_Vajpayee_0013.

###  LIVE FACE RECOGNITION

In [13]:
# ENCODINGS_FILE = "face_encodings.pkl"

# # Load encodings
# with open(ENCODINGS_FILE, "rb") as f:
#     data = pickle.load(f)

# video_capture = cv2.VideoCapture(0)

# while True:
#     ret, frame = video_capture.read()
#     if not ret:
#         print("Failed to capture image")
#         break

#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     # Detect faces
#     face_locations = face_recognition.face_locations(rgb_frame)
#     face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

#     for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
#         matches = face_recognition.compare_faces(data["encodings"], face_encoding, tolerance=0.4)
#         name = "Unknown"

#         # Find the best match
#         face_distances = face_recognition.face_distance(data["encodings"], face_encoding)
#         best_match_index = np.argmin(face_distances)

#         if matches[best_match_index]:
#             name = data["names"][best_match_index]

#         # Draw rectangle and label
#         cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
#         cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

#     # Show output
#     cv2.imshow("Face Recognition", frame)

#     # Exit on 'q'
#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break

# video_capture.release()
# cv2.destroyAllWindows()

In [20]:
ENCODINGS_FILE = "face_encodings.pkl"

# Load encodings
with open(ENCODINGS_FILE, "rb") as f:
    data = pickle.load(f)

video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to capture image")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # detect face and encode them
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        face_distances = face_recognition.face_distance(data["encodings"], face_encoding)
        best_match_index = np.argmin(face_distances)

        # set threshold = 0.4 to strict threshold for unknown faces
        threshold = 0.4
        if face_distances[best_match_index] < threshold:
            name = data["names"][best_match_index]
            confidence = 1 - face_distances[best_match_index]
        else:
            name = "Unknown"
            confidence = 0

        # Draw ractangle and label them with confidence
        label = f"{name} ({confidence:.2f})"
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()


Failed to capture image


[ WARN:0@8760.909] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@8760.909] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


### Try knn for more accuracy

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Load encoded face data
with open("face_encodings.pkl", "rb") as f:
    data = pickle.load(f)

# Train k-NN with stored face encodings
knn = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
knn.fit(data["encodings"], data["names"])

# Save the trained model
with open("knn_face_model.pkl", "wb") as f:
    pickle.dump(knn, f)

print("k-NN model trained and saved!")


In [ ]:
import cv2
import face_recognition
import pickle

# Load trained k-NN model
with open("knn_face_model.pkl", "rb") as f:
    knn = pickle.load(f)

video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Predict name using k-NN
        name = knn.predict([face_encoding])[0]

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Face Recognition with k-NN", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()

In [29]:
# import cv2
# import face_recognition
# import numpy as np
# from src.utils import load_encodings
# import pickle

def load_encodings(encodings_file):
    with open(encodings_file, "rb") as f:
        data = pickle.load(f)
    return data

class FaceDetector:
    def __init__(self, faceCascadePath):
        """Apply Haar Cascade to detect faces."""
        self.faceCascade = cv2.CascadeClassifier(faceCascadePath)
        
        # Check if the cascade is loaded properly
        if self.faceCascade.empty():
            raise ValueError(f"Error loading cascade file: {faceCascadePath}")

    def detect(self, image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)):
        """Return x, y, width, height of detected faces."""
        rects = self.faceCascade.detectMultiScale(image, scaleFactor=scaleFactor, minNeighbors=minNeighbors, minSize=minSize)
        return rects

def detect_live(encodings_file="face_encodings.pkl"):
    data = load_encodings(encodings_file)

    # Start video capture from webcam
    video_cap = cv2.VideoCapture(0)
    
    # frontal_face_Cascade_Path = "src/haarcascade_frontalface_default.xml"
    frontal_face_Cascade_Path = "/home/user/Desktop/internship project/Cyfuture-face-recognition-opencv/face-recog-env/lib/python3.13/site-packages/cv2/data/haarcascade_frontalface_default.xml"
    fd = FaceDetector(frontal_face_Cascade_Path)

    while True:
        ret, frame = video_cap.read()
        if not ret:
            print("Failed to capture image")
            break

        # Resize the image to a fixed width and height - required for linux to reshape the image as linux is not built for general purpose task. Its default image size too small.
        frame_resized = cv2.resize(frame, (640, 480))

        # Convert rgb image to grayscale image for face detection
        gray_image = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

        # Detect faces using Haar Cascade
        faces = fd.detect(gray_image)

        # Convert the frame to RGB for face recognition
        rgb_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)

        # Recognize faces
        for (x, y, w, h) in faces:
            face_image = rgb_frame[y:y+h, x:x+w]

            # Get face encodings for the detected face
            face_encodings = face_recognition.face_encodings(face_image)

            if len(face_encodings) > 0:
                face_encoding = face_encodings[0]

                # Compare face encoding with known encodings
                face_distances = face_recognition.face_distance(data["encodings"], face_encoding)
                best_match_index = np.argmin(face_distances)

                # Set a strict threshold (0.4) for recognition
                threshold = 0.4
                if face_distances[best_match_index] < threshold:
                    name = data["names"][best_match_index]
                    confidence = 1 - face_distances[best_match_index]
                else:
                    name = "Unknown"
                    confidence = 0

                # Draw rectangle and label with confidence
                label = f"{name} ({confidence:.2f})"
                cv2.rectangle(frame_resized, (x, y), (x + w, y + h), (127, 255, 0), 2)
                cv2.putText(frame_resized, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (127, 255, 0), 2)

        # Show the resized video feed
        cv2.imshow("Live Video - Face Recognition", frame_resized)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord("q") or cv2.getWindowProperty("Live Video - Face Recognition", cv2.WND_PROP_VISIBLE) < 1:
            break

    # Release resources
    video_cap.release()
    cv2.destroyAllWindows()

In [30]:
detect_live()

Failed to capture image


[ WARN:0@9079.090] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@9079.090] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
